In [3]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

mydb = mysql.connector.connect(host="localhost", user="root", password="nb24", database="nbdata")
cursor = mydb.cursor()

def login():
    username = username_entry.get()
    password = password_entry.get()

    # Connect to the MySQL database
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='nb24',
            database='nbdata'
        )
        cursor = connection.cursor()
        
        # Query to check if the username and password exist in the database
        query = "SELECT * FROM logindata WHERE username = %s AND password = %s"
        cursor.execute(query, (username, password))
        result = cursor.fetchone()
        
        if result:
            messagebox.showinfo("Login Success", "Welcome to the database visualizations!")
            open_landing_page()
        else:
            signup_prompt()

        cursor.close()
        connection.close()

    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error connecting to the database: {err}")

def signup_prompt():
    signup_window = tk.Toplevel(root)
    signup_window.title("Sign Up")

    # Center the window on the screen
    window_width = 300
    window_height = 200
    screen_width = signup_window.winfo_screenwidth()
    screen_height = signup_window.winfo_screenheight()
    position_top = int(screen_height / 2 - window_height / 2)
    position_right = int(screen_width / 2 - window_width / 2)
    signup_window.geometry(f'{window_width}x{window_height}+{position_right}+{position_top}')

    frame = tk.Frame(signup_window)
    frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

    tk.Label(frame, text="Username").grid(row=0, column=0, pady=10)
    signup_username_entry = tk.Entry(frame)
    signup_username_entry.grid(row=0, column=1, pady=10)

    tk.Label(frame, text="Password").grid(row=1, column=0, pady=10)
    signup_password_entry = tk.Entry(frame, show='*')
    signup_password_entry.grid(row=1, column=1, pady=10)

    def signup():
        username = signup_username_entry.get()
        password = signup_password_entry.get()

        try:
            connection = mysql.connector.connect(
                host='localhost',
                user='root',
                password='nb24',
                database='nbdata'
            )
            cursor = connection.cursor()

            # Query to insert the new username and password into the database
            query = "INSERT INTO logindata(username, password) VALUES (%s, %s)"
            cursor.execute(query, (username, password))
            connection.commit()

            cursor.close()
            connection.close()

            messagebox.showinfo("Sign Up Success", "You have successfully signed up! Please log in.")
            signup_window.destroy()

        except mysql.connector.Error as err:
            messagebox.showerror("Error", f"Error connecting to the database: {err}")

    signup_button = tk.Button(frame, text="Sign Up", command=signup)
    signup_button.grid(row=2, columnspan=2, pady=10)

def open_landing_page():
    landing_page = tk.Toplevel(root)
    landing_page.title("Landing Page")

    # Make the landing page full screen
    landing_page.attributes('-fullscreen', True)

    tk.Label(landing_page, text="Face-Recognition Visuals!...", font=("Helvetica", 16)).pack(pady=20)

    data = "SELECT * FROM collecting_data"
    cursor.execute(data)
    
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    
    male = df[df['Gender']=='Male']
    female = df[df['Gender']=='Female']
    
    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    df['Age_int'] = label_encoder.fit_transform(df['Age'])
    label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
    
    label_encoder = LabelEncoder()
    male['Male_Age'] = label_encoder.fit_transform(male['Age'])
    
    label_encoder = LabelEncoder()
    female['Female_Age'] = label_encoder.fit_transform(female['Age'])

    # Create first sample plot
    fig1 = Figure(figsize=(3, 2), dpi=100)
    plot1 = fig1.add_subplot(111)
    plot1.bar(df['Gender'], df['Age'], label='Age by Gender', color='hotpink')
    plot1.set_title('Age-wise Male & Female')
    plot1.legend()

    # Embed the first plot in the Tkinter window
    canvas1 = FigureCanvasTkAgg(fig1, master=landing_page)
    canvas1.draw()
    canvas1.get_tk_widget().place(x=50, y=50, width=600, height=400)

    # Create second sample plot with seaborn
    fig2 = Figure(figsize=(4, 3), dpi=100)
    plot2 = fig2.add_subplot(111)
    sns.barplot(x='Gender', y='Age_int', hue='Age_int', data=df, ax=plot2)
    plot2.set_title('Age-wise Male & Female')
    plot2.legend()

    # Embed the second plot in the Tkinter window
    canvas2 = FigureCanvasTkAgg(fig2, master=landing_page)
    canvas2.draw()
    canvas2.get_tk_widget().place(x=50, y=500, width=1600, height=400)

    # Create third sample plot
    fig3 = Figure(figsize=(3, 2), dpi=100)
    plot3 = fig3.add_subplot(111)
    plot3.plot(male['Male_Age'], label="Male", linestyle='dotted', marker="o")
    plot3.plot(female['Female_Age'], label="Female", marker='d')
    plot3.set_title(' Difference b/t Male_Age & Female_Age')
    plot3.legend()
    plot3.grid()

    # Embed the third plot in the Tkinter window
    canvas3 = FigureCanvasTkAgg(fig3, master=landing_page)
    canvas3.draw()
    canvas3.get_tk_widget().place(x=900, y=50, width=600, height=400)

    tk.Button(landing_page, text="Logout", command=landing_page.destroy).place(x=750, y=950)

root = tk.Tk()
root.title("Login Form")

# Center the window on the screen
window_width = 300
window_height = 200
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
position_top = int(screen_height / 2 - window_height / 2)
position_right = int(screen_width / 2 - window_width / 2)
root.geometry(f'{window_width}x{window_height}+{position_right}+{position_top}')

# Create a frame for the login form
frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

# Username label and entry
username_label = tk.Label(frame, text="Username")
username_label.grid(row=0, column=0, pady=10)
username_entry = tk.Entry(frame)
username_entry.grid(row=0, column=1, pady=10)

# Password label and entry
password_label = tk.Label(frame, text="Password")
password_label.grid(row=1, column=0, pady=10)
password_entry = tk.Entry(frame, show='*')
password_entry.grid(row=1, column=1, pady=10)

# Login button
login_button = tk.Button(frame, text="Login", command=login)
login_button.grid(row=2, columnspan=2, pady=10)

root.mainloop()


In [ ]:
pip install scikit-learn